In [1]:
import boto3

In [2]:
email = "christopherkelly@engi.network"
from_addr = email

In [3]:
contact_list = "engi-newsletter"
template_name = "engi-newsletter-welcome-template"

In [4]:
client = boto3.client("sesv2")

### Celery SQS backend

See https://docs.celeryq.dev/en/stable/getting-started/backends-and-brokers/sqs.html#broker-sqs

Be careful about having multiple Celery workers dequeueing jobs from the same queue!

In [5]:
sqs_client = boto3.client("sqs")

In [6]:
sqs_client.list_queues()

{'QueueUrls': ['https://us-west-2.queue.amazonaws.com/163803973373/celery',
  'https://us-west-2.queue.amazonaws.com/163803973373/dev-email-server-celery',
  'https://us-west-2.queue.amazonaws.com/163803973373/e969e804-d411-48ef-b050-df6afcced3da-same-story-test-queue',
  'https://us-west-2.queue.amazonaws.com/163803973373/prod-email-server-celery',
  'https://us-west-2.queue.amazonaws.com/163803973373/same-story-api',
  'https://us-west-2.queue.amazonaws.com/163803973373/same-story-check-queue'],
 'ResponseMetadata': {'RequestId': 'd646e254-f674-5b63-a3a4-b8a7bde16f0c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd646e254-f674-5b63-a3a4-b8a7bde16f0c',
   'date': 'Mon, 09 May 2022 17:03:17 GMT',
   'content-type': 'text/xml',
   'content-length': '825'},
  'RetryAttempts': 0}}

In [12]:
sqs_client.delete_queue(QueueUrl="https://us-west-2.queue.amazonaws.com/163803973373/celery-email-server-devcelery")

{'ResponseMetadata': {'RequestId': 'd2087a39-6373-51b8-a351-2086db9ae309',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd2087a39-6373-51b8-a351-2086db9ae309',
   'date': 'Thu, 31 Mar 2022 14:11:53 GMT',
   'content-type': 'text/xml',
   'content-length': '211'},
  'RetryAttempts': 0}}

In [5]:
topics = [
    {
        'TopicName': 'engi-programmer',
        'DisplayName': 'Engi Programmers',
        'Description': 'updates of specific interest for programmers',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
    {
        'TopicName': 'engi-business',
        'DisplayName': 'Engi Business',
        'Description': 'updates of specific interest for business',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
    {
        'TopicName': 'engi-investor',
        'DisplayName': 'Engi Investor',
        'Description': 'updates of specific interest for investors',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
    {
        'TopicName': 'engi-curious',
        'DisplayName': 'Engi Curious',
        'Description': 'updates of interest for the curious',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
    {
        'TopicName': 'engi-newsletter',
        'DisplayName': 'The Engi Newsletter',
        'Description': 'subscribe to updates about the development of engi',
        'DefaultSubscriptionStatus': 'OPT_IN'
    },
]

### Templates

In [5]:
client.list_email_templates()

{'ResponseMetadata': {'RequestId': '0d86d247-1f9e-40ab-a95f-71386d04c232',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 10 Mar 2022 00:46:40 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0d86d247-1f9e-40ab-a95f-71386d04c232'},
  'RetryAttempts': 0},
 'TemplatesMetadata': [{'TemplateName': 'engi-newsletter-welcome-template',
   'CreatedTimestamp': datetime.datetime(2021, 10, 10, 12, 59, 20, tzinfo=tzlocal())}]}

In [6]:
client.get_email_template(TemplateName=template_name)

{'ResponseMetadata': {'RequestId': '24f5b140-82b3-4e4e-81bd-945a65964423',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 30 Mar 2022 23:51:08 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '235',
   'connection': 'keep-alive',
   'x-amzn-requestid': '24f5b140-82b3-4e4e-81bd-945a65964423'},
  'RetryAttempts': 0},
 'TemplateName': 'engi-newsletter-welcome-template',
 'TemplateContent': {'Subject': 'Welcome to Engi',
  'Text': 'Hi friend!\nThanks for subscribing to Engi updates.',
  'Html': '<h1>Hi friend!</h1><p>Thanks for subscribing to Engi updates.</p>'}}

In [7]:
client.update_email_template(
    TemplateName=template_name,
    TemplateContent={
        'Subject': 'Welcome to Engi',
        'Text': 'Hi friend!\nThanks for subscribing to Engi updates.',
        'Html': '<h1>Hi friend!</h1><p>Thanks for subscribing to Engi updates.</p>'
    }
)

{'ResponseMetadata': {'RequestId': '5a77b059-c6f6-4b64-a738-637fe57c375b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 30 Mar 2022 23:37:34 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5a77b059-c6f6-4b64-a738-637fe57c375b'},
  'RetryAttempts': 0}}

### Contact Lists

In [7]:
client.list_contact_lists()

{'ResponseMetadata': {'RequestId': 'fa3fd31e-1165-43c7-acc2-092bdb907eca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 09 May 2022 17:03:36 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'fa3fd31e-1165-43c7-acc2-092bdb907eca'},
  'RetryAttempts': 0},
 'ContactLists': [{'ContactListName': 'engi-newsletter',
   'LastUpdatedTimestamp': datetime.datetime(2022, 3, 11, 6, 13, 22, 579000, tzinfo=tzlocal())}]}

In [5]:
client.get_contact_list(ContactListName=contact_list)

{'ResponseMetadata': {'RequestId': '9d65641b-ae7b-4e5a-acfd-6d6340a63fe4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 17 May 2022 13:05:08 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '981',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9d65641b-ae7b-4e5a-acfd-6d6340a63fe4'},
  'RetryAttempts': 0},
 'ContactListName': 'engi-newsletter',
 'Topics': [{'TopicName': 'engi-programmer',
   'DisplayName': 'Engi Programmers',
   'Description': 'updates of specific interest for programmers',
   'DefaultSubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-business',
   'DisplayName': 'Engi Business',
   'Description': 'updates of specific interest for business',
   'DefaultSubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-investor',
   'DisplayName': 'Engi Investor',
   'Description': 'updates of specific interest for investors',
   'DefaultSubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-curious',
   'DisplayName': 'Engi Curious',


In [9]:
r = client.create_contact_list(
    ContactListName=contact_list,
    Topics=topics,
    Description="testing",
)

BadRequestException: An error occurred (BadRequestException) when calling the CreateContactList operation: A maximum of 1 Lists allowed per account.

In [6]:
client.list_contacts(ContactListName=contact_list)

{'ResponseMetadata': {'RequestId': 'c7f0ce58-0527-4919-9de2-e95882dfab63',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 17 May 2022 13:05:19 GMT',
   'content-type': 'application/json',
   'content-length': '2332',
   'connection': 'close',
   'x-amzn-requestid': 'c7f0ce58-0527-4919-9de2-e95882dfab63'},
  'RetryAttempts': 0},
 'Contacts': [{'EmailAddress': 'garrett.maring@gmail.com',
   'TopicPreferences': [{'TopicName': 'engi-investor',
     'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-newsletter', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-programmer', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-curious', 'SubscriptionStatus': 'OPT_IN'},
    {'TopicName': 'engi-business', 'SubscriptionStatus': 'OPT_IN'}],
   'UnsubscribeAll': False,
   'LastUpdatedTimestamp': datetime.datetime(2022, 3, 30, 17, 48, 45, 30000, tzinfo=tzlocal())},
  {'EmailAddress': 'cyung10892@gmail.com',
   'TopicPreferences': [{'TopicName': 'engi-investor',
     'Sub

In [11]:
topic = "engi-newsletter"
r = client.list_contacts(
    ContactListName=contact_list,
    Filter={
        'FilteredStatus': 'OPT_IN',
        'TopicFilter': {
            'TopicName': 'engi-newsletter',
            'UseDefaultIfPreferenceUnavailable': True
        }
    },
    PageSize=10,
); r

{'ResponseMetadata': {'RequestId': '00c7113d-4248-4d1f-90f8-fe6ea4c5b4ef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 10 Mar 2022 00:57:18 GMT',
   'content-type': 'application/json',
   'content-length': '3063',
   'connection': 'close',
   'x-amzn-requestid': '00c7113d-4248-4d1f-90f8-fe6ea4c5b4ef'},
  'RetryAttempts': 0},
 'Contacts': [{'EmailAddress': '12@engi.network',
   'TopicPreferences': [{'TopicName': 'engi-newsletter',
     'SubscriptionStatus': 'OPT_IN'}],
   'UnsubscribeAll': False,
   'LastUpdatedTimestamp': datetime.datetime(2022, 3, 9, 13, 46, 45, 86000, tzinfo=tzlocal())},
  {'EmailAddress': '3@engi.network',
   'TopicPreferences': [{'TopicName': 'engi-newsletter',
     'SubscriptionStatus': 'OPT_IN'}],
   'UnsubscribeAll': False,
   'LastUpdatedTimestamp': datetime.datetime(2022, 3, 9, 13, 46, 44, 276000, tzinfo=tzlocal())},
  {'EmailAddress': '18@engi.network',
   'TopicPreferences': [{'TopicName': 'engi-newsletter',
     'SubscriptionStatus': 'OPT_IN'}]

In [14]:
r = client.list_contacts(ContactListName=contact_list, NextToken=r["NextToken"])

In [8]:
r = client.update_contact_list(
    ContactListName=contact_list,
    Topics=topics,
)

In [9]:
r

{'ResponseMetadata': {'RequestId': 'aac878c3-e398-487b-a1a4-36e81891dad2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 11 Mar 2022 14:13:22 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'aac878c3-e398-487b-a1a4-36e81891dad2'},
  'RetryAttempts': 0}}

### Contact Management

In [10]:
client.create_contact(
    ContactListName=contact_list,
    EmailAddress=email
)

{'ResponseMetadata': {'RequestId': '63f3d541-d178-4c48-a1f5-aaa80d8ead11',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 09 Mar 2022 16:13:05 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '63f3d541-d178-4c48-a1f5-aaa80d8ead11'},
  'RetryAttempts': 0}}

In [9]:
r = client.get_contact(
    ContactListName=contact_list,
    EmailAddress=email
)#; r

In [10]:
r

{'ResponseMetadata': {'RequestId': '6baa36d5-b641-4211-9aa0-0fbc4cade675',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 17 May 2022 13:06:36 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '560',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6baa36d5-b641-4211-9aa0-0fbc4cade675'},
  'RetryAttempts': 0},
 'ContactListName': 'engi-newsletter',
 'EmailAddress': 'christopherkelly@engi.network',
 'TopicPreferences': [{'TopicName': 'engi-investor',
   'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-newsletter', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-programmer', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-curious', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-business', 'SubscriptionStatus': 'OPT_IN'}],
 'UnsubscribeAll': False,
 'AttributesData': '{}',
 'CreatedTimestamp': datetime.datetime(2022, 5, 17, 6, 6, 23, 159000, tzinfo=tzlocal()),
 'LastUpdatedTimestamp': datetime.datetime(2022, 5, 17, 6, 6

In [13]:
r = client.update_contact(
    ContactListName=contact_list,
    EmailAddress=email,
    TopicPreferences=[
        {
            'TopicName': 'engi-investor',
            'SubscriptionStatus': 'OPT_IN'
        },
    ],
    UnsubscribeAll=False,
    AttributesData='{"name": "chris", "favoriteanimal": "bonobo"}'
)


In [19]:
email = "g@engi.network"

In [15]:
r = client.get_contact(
    ContactListName=contact_list,
    EmailAddress=email
); r

{'ResponseMetadata': {'RequestId': 'e1a06972-6c43-42c8-8082-e0c097e11e71',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 11 Mar 2022 16:47:55 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '612',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e1a06972-6c43-42c8-8082-e0c097e11e71'},
  'RetryAttempts': 0},
 'ContactListName': 'engi-newsletter',
 'EmailAddress': 'christopherkelly@engi.network',
 'TopicPreferences': [{'TopicName': 'engi-investor',
   'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-newsletter', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-programmer', 'SubscriptionStatus': 'OPT_IN'},
  {'TopicName': 'engi-curious', 'SubscriptionStatus': 'OPT_OUT'},
  {'TopicName': 'engi-business', 'SubscriptionStatus': 'OPT_IN'}],
 'UnsubscribeAll': False,
 'AttributesData': '{"name": "chris", "favoriteanimal": "bonobo"}',
 'CreatedTimestamp': datetime.datetime(2022, 3, 11, 8, 46, 48, 995000, tzinfo=tzlocal()),
 'LastUpdatedTi

In [16]:
[d["TopicName"] for d in r["TopicPreferences"]]

['engi-newsletter']

In [18]:
r["ResponseMetadata"]["HTTPStatusCode"]

200

In [21]:
r = client.delete_contact(
    ContactListName=contact_list,
    EmailAddress=email
); r

{'ResponseMetadata': {'RequestId': '2537df60-c3b2-4065-bc78-3818f725b131',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 09 Mar 2022 18:49:37 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2537df60-c3b2-4065-bc78-3818f725b131'},
  'RetryAttempts': 0}}

### Sending Email

In [7]:
from_addr = "g@engi.network"
to_addr = "christopherkelly@engi.network"

#### SendEmail

In [7]:
r = client.send_email(
    FromEmailAddress=from_addr,
    Destination={
        'ToAddresses': [
            to_addr,
        ],
    },
    Content={
        'Simple': {
            'Subject': {
                'Data': 'test 1',
                'Charset': 'UTF-8'
            },
            'Body': {
                'Text': {
                    'Data': 'body',
                    'Charset': 'UTF-8'
                },
                'Html': {
                    'Data': '<p>body</p>',
                    'Charset': 'UTF-8'
                }
            }
        },


    },
    ListManagementOptions={
        'ContactListName': contact_list,
        'TopicName': contact_list
    }
)

In [ ]:
# use send_bulk_email

In [15]:
template_name

'engi-newsletter-welcome-template'

In [19]:
%pdb

Automatic pdb calling has been turned OFF


In [12]:
to_addr = email

In [16]:
contact_list

'engi-newsletter-test'

In [14]:
r = client.send_email(
        FromEmailAddress=from_addr,
        Destination={
            "ToAddresses": [
                to_addr,
            ],
        },
        Content={
            "Template": {
                "TemplateName": template_name,
                "TemplateData": "{}",
            },
        },
        ListManagementOptions={
            "ContactListName": contact_list,
            "TopicName": contact_list,
        }
)

NotFoundException: An error occurred (NotFoundException) when calling the SendEmail operation: List with name: engi-newsletter-test doesn't exist.

In [ ]:
r = client.send_email(
    FromEmailAddress=from_addr,
    Destination={
        'ToAddresses': [
            to_addr,
        ],
    },
    Content={
        'Template': {
            'TemplateName': template_name,
            'TemplateData': '{"name": "chris", "favoriteanimal": "bonobo"}'
        },
    },
    ListManagementOptions={
        'ContactListName': contact_list,
        'TopicName': contact_list
    }
)

In [ ]:
r

{'ResponseMetadata': {'RequestId': '60866212-76a5-4d86-9fce-7157f48f0e0b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 10 Mar 2022 00:51:27 GMT',
   'content-type': 'application/json',
   'content-length': '76',
   'connection': 'keep-alive',
   'x-amzn-requestid': '60866212-76a5-4d86-9fce-7157f48f0e0b'},
  'RetryAttempts': 0},
 'MessageId': '0101017f714f27f4-74722b9e-a792-4ca1-bbeb-311a5e4c8bcd-000000'}

#### SendBulkEmail

In [41]:
contacts = list_contacts(client, contact_list, topic)

In [48]:
entries = [e for e in get_bulk_entries(contacts) if "kelly" in e["Destination"]["ToAddresses"][0]]

In [49]:
r = client.send_bulk_email(
    FromEmailAddress=from_addr,
    DefaultContent={
        'Template': {
            'TemplateName': template_name,
            'TemplateData': '{"name": "friend", "favoriteanimal": "not sure"}'
        }
    },
    BulkEmailEntries=entries,
); r

{'ResponseMetadata': {'RequestId': 'ed137a45-d649-4bf9-9eb0-4f4a751f5159',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 10 Mar 2022 01:48:43 GMT',
   'content-type': 'application/json',
   'content-length': '136',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ed137a45-d649-4bf9-9eb0-4f4a751f5159'},
  'RetryAttempts': 0},
 'BulkEmailEntryResults': [{'Status': 'SUCCESS',
   'MessageId': '0101017f718394bf-628b0eb5-cdce-4685-a721-ca31e611151b-000000'}]}

In [2]:
results = [{'Status': 'SUCCESS',
   'MessageId': '0101017f718394bf-628b0eb5-cdce-4685-a721-ca31e611151b-000000'}]

In [7]:
all([k["Status"] == "SUCCESS" for k in results])

True

In [8]:
r

{'ResponseMetadata': {'RequestId': '9dba4950-a902-44bf-96c8-d93ff6e3b34c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 09 Mar 2022 19:09:26 GMT',
   'content-type': 'application/json',
   'content-length': '76',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9dba4950-a902-44bf-96c8-d93ff6e3b34c'},
  'RetryAttempts': 0},
 'MessageId': '0101017f701607b8-0f106dd6-8da9-4405-89d7-bce6cb37cc50-000000'}

### Scratch Pad

In [9]:
dir(client.exceptions)

['AccountSuspendedException',
 'AlreadyExistsException',
 'BadRequestException',
 'ClientError',
 'ConcurrentModificationException',
 'ConflictException',
 'InvalidNextTokenException',
 'LimitExceededException',
 'MailFromDomainNotVerifiedException',
 'MessageRejected',
 'NotFoundException',
 'SendingPausedException',
 'TooManyRequestsException',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_code_to_exception',
 'from_code']

In [7]:
s = "{ \"name\":\"Alejandro\", \"favoriteanimal\": \"alligator\" }"

In [9]:
import json

In [11]:
json.loads(s)

{'name': 'Alejandro', 'favoriteanimal': 'alligator'}

In [14]:
json.dumps(json.dumps(_11))

'"{\\"name\\": \\"Alejandro\\", \\"favoriteanimal\\": \\"alligator\\"}"'

In [10]:
session = boto3.Session()

In [13]:
sqs_client = boto3.client("sqs")

In [15]:
sqs_client.list_queues()

{'ResponseMetadata': {'RequestId': 'e734cb36-9582-5571-84ea-9192a0d94383',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e734cb36-9582-5571-84ea-9192a0d94383',
   'date': 'Thu, 10 Mar 2022 17:19:23 GMT',
   'content-type': 'text/xml',
   'content-length': '228'},
  'RetryAttempts': 0}}

In [1]:
import pycurl

In [11]:
session.get_available_services()

['accessanalyzer',
 'account',
 'acm',
 'acm-pca',
 'alexaforbusiness',
 'amp',
 'amplify',
 'amplifybackend',
 'amplifyuibuilder',
 'apigateway',
 'apigatewaymanagementapi',
 'apigatewayv2',
 'appconfig',
 'appconfigdata',
 'appflow',
 'appintegrations',
 'application-autoscaling',
 'application-insights',
 'applicationcostprofiler',
 'appmesh',
 'apprunner',
 'appstream',
 'appsync',
 'athena',
 'auditmanager',
 'autoscaling',
 'autoscaling-plans',
 'backup',
 'backup-gateway',
 'batch',
 'braket',
 'budgets',
 'ce',
 'chime',
 'chime-sdk-identity',
 'chime-sdk-meetings',
 'chime-sdk-messaging',
 'cloud9',
 'cloudcontrol',
 'clouddirectory',
 'cloudformation',
 'cloudfront',
 'cloudhsm',
 'cloudhsmv2',
 'cloudsearch',
 'cloudsearchdomain',
 'cloudtrail',
 'cloudwatch',
 'codeartifact',
 'codebuild',
 'codecommit',
 'codedeploy',
 'codeguru-reviewer',
 'codeguruprofiler',
 'codepipeline',
 'codestar',
 'codestar-connections',
 'codestar-notifications',
 'cognito-identity',
 'cognito-i